# Laboratorium 6


In [84]:
from collections import deque
from PIL import Image
from time import perf_counter
AdjList=[]

def init_trie(keywords):
    AdjList=[]
    create_empty_trie()
    add_keywords(keywords)
    set_fail_transitions()
    
def create_empty_trie():
    AdjList.append({'value':'','next_states':[],'fail_state':0,'output':[]})

def add_keywords(keywords):
    for word in keywords:
        add_keyword(word)

def find_next_state(curr_state,val):
    for node in AdjList[curr_state]['next_states']:
        if AdjList[node]['value']==val:
            return node
    return None
def add_keyword(keyword):
    current_state = 0
    j = 0
    child = find_next_state(current_state, keyword[j])
    while child != None:
        current_state = child
        j = j + 1
        if j < len(keyword):
            child = find_next_state(current_state, keyword[j])
        else:
            break
    for i in range(j, len(keyword)):
        node = {'value':keyword[i],'next_states':[],'fail_state':0,'output':[]}
        AdjList.append(node)
        AdjList[current_state]["next_states"].append(len(AdjList) - 1)
        current_state = len(AdjList) - 1
    AdjList[current_state]["output"].append(keyword)
    
def set_fail_transitions():
    q = deque()
    child = 0
    for node in AdjList[0]["next_states"]:
        q.append(node)
        AdjList[node]["fail_state"] = 0
    while q:
        r = q.popleft()
        for child in AdjList[r]["next_states"]:
            q.append(child)
            state = AdjList[r]["fail_state"]
            while find_next_state(state, AdjList[child]["value"]) == None \
    and state != 0:
                state = AdjList[state]["fail_state"]
            AdjList[child]["fail_state"] = find_next_state(state, 
    AdjList[child]["value"])
            if AdjList[child]["fail_state"] is None:
                AdjList[child]["fail_state"] = 0
            AdjList[child]["output"] = AdjList[child]["output"] + AdjList[AdjList[child]["fail_state"]]["output"]
            
def get_keywords_found(line):
#     line = line.lower()
    current_state = 0
    keywords_found = []
    states=[]
    for i in range(len(line)):
        while find_next_state(current_state, line[i]) is None and current_state != 0:
            current_state = AdjList[current_state]["fail_state"]
        current_state = find_next_state(current_state, line[i])
        if current_state is None:
            current_state = 0
        else:
            for j in AdjList[current_state]["output"]:
                keywords_found.append({"index":i-len(j) + 1,"word":j})
        states.append(current_state)
        
    return states

In [85]:
pat=['aab','bab','aab']
P=[
    'abababb',
    'aaaabbb',
    'bbbaaab',
    'aaabbaa',
    'bbaaabb',
    'aabaaaa',
   ]

In [86]:
def prepare_pattern_kmp(pattern):
    """function return list containing values 
    of prefix function for each position"""
    pi=[0]*len(pattern)
    for i in range(1,len(pi)):
        j=pi[i-1]
        while j>0 and pattern[i]!=pattern[j]:
            j=pi[j-1]
        if pattern[i]==pattern[j]:
            j+=1
        pi[i]=j
    return pi
def kmp(pattern,text,pi):
    """function returns list containing postions of pattern in text
        pi is list of values of prefix functions"""
    q = 0
    ans = []
    for i in range(0, len(text)):
        while(q > 0 and pattern[q] != text[i]):
            q = pi[q-1]
        if(pattern[q] == text[i]):
            q = q + 1
        if(q == len(pattern)):
            ans.append(i+1-q)
            q = pi[q-1]
    return ans

def match_2drect(pattern,P):
    t0=perf_counter()
    init_trie(pattern)
    patterns=[get_keywords_found(s)[-1] for s in pattern]
    t1=perf_counter()
    print('time spending on constructing automata: ',t1-t0)
    T=[[0 for i in range(len(P[0]))] for i in range(len(P))]
    for i in range(len(P[0])):
        s=[]
        for j in range(len(P)):
            s.append(P[j][i])
        res=get_keywords_found(s)
        for j,w in enumerate(res):
            T[j][i]=w
    pi=prepare_pattern_kmp(patterns)
    res=[]
    for i in range(len(T)):
        positions=kmp(patterns,T[i],pi)
        if len(positions)!=0:
            res.append([(i,p) for p in positions])
    return res
print(match_2drect(pat,P))

time spending on constructing automata:  0.00010516499969526194
[[(2, 0)]]


Litera x pod literą x

In [68]:
max_line_len=0
lines=[]
#fix it
with open('haystack.txt','r') as file:
    lines=file.readlines()
    max_line_len=max(len(l) for l in lines)
    for i in range(len(lines)):
        lines[i]+=' '*(max_line_len-len(lines[i]))
res=match_2drect(['aa'],lines)
for l in [chr(x) for x in range(97, 123)]:
    print('letter',l)
    res=match_2drect([l+l],lines)
    print(res)

time spending on constructing automata:  0.0001052980005624704
letter a
time spending on constructing automata:  3.072400068049319e-05
[[(1, 82)], [(4, 30)], [(6, 60)], [(7, 63)], [(21, 6)], [(29, 69)], [(32, 50), (32, 73)], [(34, 66)], [(38, 4)], [(53, 12)], [(54, 12), (54, 48)], [(57, 11)], [(58, 36)], [(59, 36)], [(60, 24)], [(65, 2), (65, 14), (65, 22)], [(66, 35)], [(70, 35)], [(77, 21), (77, 74)], [(78, 42), (78, 61)], [(79, 59)], [(80, 37)]]
letter b
time spending on constructing automata:  2.884900095523335e-05
[]
letter c
time spending on constructing automata:  2.8357999326544814e-05
[[(4, 54)], [(11, 45)], [(14, 10)], [(42, 0)], [(69, 0)], [(83, 41)]]
letter d
time spending on constructing automata:  3.666999873530585e-05
[[(38, 19)]]
letter e
time spending on constructing automata:  3.376399945409503e-05
[[(1, 63)], [(2, 8)], [(5, 77)], [(8, 65)], [(11, 1), (11, 64)], [(15, 2)], [(16, 43)], [(18, 6)], [(19, 27)], [(21, 10)], [(22, 61)], [(23, 53)], [(25, 3), (25, 65)], [(29

th

In [69]:
res=match_2drect(['tt','hh'],lines)
print(res)


time spending on constructing automata:  0.0004779720002261456
[]


t h

In [70]:
res=match_2drect(['tt','  ','hh'],lines)
print(res)

time spending on constructing automata:  0.0021198169997660443
[[(38, 0)]]


image

In [71]:

from matplotlib.image import imread
import numpy as np

def img_to_matrix(image_path):
    with Image.open(image_path) as img:
        pixel_map = img.load()
        pixels = []
        for row in range(img.height):
            pix = []
            for col in range(img.width):
                pix.append(pixel_map[col, row][0])
            pixels.append(pix)
        return pixels
def transpose(matrix):
    return np.array(matrix).T.tolist()

Literka i

In [72]:


pattern=transpose(img_to_matrix('patterns/i.png'))
pat=img_to_matrix('haystack.png')
print(len(pat))
res=match_2drect(pattern,pat)
print(res)

1900
time spending on constructing automata:  0.00031646400020690635
[[(46, 135), (46, 429), (46, 505), (46, 641), (46, 674)], [(68, 71), (68, 197), (68, 350), (68, 455), (68, 672)], [(90, 76), (90, 538)], [(112, 26), (112, 375), (112, 523), (112, 676)], [(134, 66), (134, 126), (134, 136), (134, 148), (134, 296), (134, 346)], [(156, 213), (156, 440), (156, 643)], [(178, 125), (178, 140), (178, 367), (178, 478), (178, 654)], [(200, 26), (200, 102), (200, 203), (200, 361), (200, 539)], [(222, 53), (222, 63), (222, 75), (222, 204), (222, 327), (222, 480), (222, 554), (222, 585)], [(244, 378), (244, 517), (244, 573)], [(266, 222), (266, 344), (266, 431), (266, 579)], [(288, 74), (288, 156), (288, 174), (288, 280), (288, 302), (288, 350), (288, 428)], [(332, 108), (332, 278), (332, 323), (332, 405), (332, 612), (332, 669)], [(354, 164), (354, 240), (354, 451)], [(376, 97), (376, 148), (376, 494), (376, 527), (376, 642), (376, 675)], [(398, 267), (398, 300), (398, 432), (398, 472), (398, 607

Literka m

In [73]:
spattern=transpose(img_to_matrix('patterns/m.png'))
pat=img_to_matrix('haystack.png')
print(len(pat))
res=match_2drect(pattern,pat)
print(res)

1900
time spending on constructing automata:  0.0002724289988691453
[[(46, 135), (46, 429), (46, 505), (46, 641), (46, 674)], [(68, 71), (68, 197), (68, 350), (68, 455), (68, 672)], [(90, 76), (90, 538)], [(112, 26), (112, 375), (112, 523), (112, 676)], [(134, 66), (134, 126), (134, 136), (134, 148), (134, 296), (134, 346)], [(156, 213), (156, 440), (156, 643)], [(178, 125), (178, 140), (178, 367), (178, 478), (178, 654)], [(200, 26), (200, 102), (200, 203), (200, 361), (200, 539)], [(222, 53), (222, 63), (222, 75), (222, 204), (222, 327), (222, 480), (222, 554), (222, 585)], [(244, 378), (244, 517), (244, 573)], [(266, 222), (266, 344), (266, 431), (266, 579)], [(288, 74), (288, 156), (288, 174), (288, 280), (288, 302), (288, 350), (288, 428)], [(332, 108), (332, 278), (332, 323), (332, 405), (332, 612), (332, 669)], [(354, 164), (354, 240), (354, 451)], [(376, 97), (376, 148), (376, 494), (376, 527), (376, 642), (376, 675)], [(398, 267), (398, 300), (398, 432), (398, 472), (398, 607)

Literka p

In [74]:
pattern=transpose(img_to_matrix('patterns/p.png'))
pat=img_to_matrix('haystack.png')
print(len(pat))
res=match_2drect(pattern,pat)
print(res)

1900
time spending on constructing automata:  0.0008275500003946945
[[(50, 158), (50, 369), (50, 556)], [(72, 235)], [(116, 455)], [(138, 26), (138, 301), (138, 657)], [(160, 383), (160, 509)], [(204, 135), (204, 445)], [(226, 136)], [(248, 340)], [(270, 255), (270, 507)], [(292, 382)], [(336, 198), (336, 484)], [(358, 667)], [(380, 168), (380, 447), (380, 615), (380, 626)], [(402, 359)], [(424, 135)], [(446, 145)], [(490, 111), (490, 184), (490, 381)], [(512, 428)], [(534, 214)], [(556, 242), (556, 638)], [(600, 142), (600, 342)], [(644, 248)], [(666, 125), (666, 207), (666, 684)], [(688, 262), (688, 329), (688, 671)], [(732, 344), (732, 446)], [(776, 35), (776, 139)], [(798, 114), (798, 328)], [(820, 44), (820, 306), (820, 543), (820, 686)], [(842, 334), (842, 345)], [(864, 620)], [(886, 302)], [(908, 26)], [(930, 76), (930, 430)], [(952, 137), (952, 199), (952, 409)], [(974, 180), (974, 191), (974, 407), (974, 634)], [(996, 71), (996, 178), (996, 492), (996, 622), (996, 633)], [(104

Lierka s

In [75]:
pattern=transpose(img_to_matrix('patterns/s.png'))
pat=img_to_matrix('haystack.png')
res=match_2drect(pattern,pat)
print(res)

time spending on constructing automata:  0.0012663119996432215
[[(46, 126), (46, 185), (46, 391), (46, 585), (46, 679), (46, 773)], [(68, 154), (68, 202), (68, 355)], [(90, 188), (90, 264), (90, 463), (90, 606)], [(112, 31), (112, 257), (112, 505), (112, 514), (112, 564), (112, 584), (112, 629)], [(134, 164), (134, 438), (134, 508), (134, 528), (134, 573), (134, 599), (134, 707), (134, 716)], [(156, 81), (156, 445), (156, 659)], [(178, 258), (178, 408), (178, 597), (178, 635)], [(200, 185), (200, 194), (200, 391), (200, 654)], [(222, 91), (222, 186), (222, 195), (222, 273), (222, 521), (222, 559), (222, 642)], [(244, 144), (244, 229), (244, 558), (244, 578), (244, 609)], [(266, 162), (266, 184), (266, 329), (266, 560)], [(288, 54), (288, 179), (288, 670)], [(332, 99), (332, 300), (332, 410), (332, 674)], [(354, 37), (354, 140), (354, 149), (354, 456), (354, 714)], [(376, 61), (376, 153), (376, 259), (376, 399), (376, 549)], [(398, 26), (398, 46), (398, 305), (398, 477), (398, 492), (39

napis pattern

In [76]:
pattern=transpose(img_to_matrix('patterns/pattern.png'))
pat=img_to_matrix('haystack.png')
res=match_2drect(pattern,pat)
print(res)

time spending on constructing automata:  0.007807382999089896
[[(490, 184)], [(512, 428)], [(556, 242)], [(600, 142)], [(644, 248)]]


Duży wzorzec

In [77]:
pattern=transpose(img_to_matrix('patterns/haystack_big.png'))
pat=img_to_matrix('haystack.png')
t0=perf_counter()
res=match_2drect(pattern,pat)
t1=perf_counter()
print(res)
print('time spend: ',t1-t0)

time spending on constructing automata:  0.6907386310012953
[[(425, 22)]]
time spend:  40.04246444000091


Średni wzorzec

In [87]:
pattern=transpose(img_to_matrix('patterns/haystack_medium.png'))
pat=img_to_matrix('haystack.png')
t0=perf_counter()
res=match_2drect(pattern,pat)
t1=perf_counter()
print(res)
print('time spend: ',t1-t0)

time spending on constructing automata:  0.07400000100096804
[[(360, 22)]]
time spend:  32.29946614800065


Mały wzorzec

In [88]:
pattern=transpose(img_to_matrix('patterns/haystack_small.png'))
pat=img_to_matrix('haystack.png')
t0=perf_counter()
res=match_2drect(pattern,pat)
t1=perf_counter()
print(res)
print('time spend: ',t1-t0)

time spending on constructing automata:  0.03146751800159109
[]
time spend:  47.14596595799958


Podział na 2,4,8 części

In [53]:
def divide_pat(n_of_div):
    matrix=img_to_matrix('haystack.png')
    print(len(matrix[0]))
    l=len(matrix[0])//n_of_div
    parts=[]
    for i in range(n_of_div):
        parts.append(matrix[:][i*l:(i+1)*l])
    return parts

In [60]:
for n_of_parts in [2,4,8]:
    parts=divide_pat(n_of_parts)
    t0=perf_counter()
    for part in parts:
        pattern=transpose(img_to_matrix('patterns/haystack_small.png'))
        res=match_2drect(pattern,part)
    t1=perf_counter()
    print('time spend while search on ',n_of_parts,'parts:' ,t1-t0)

860
time spending on constructing automata:  0.0026459549990249798
time spending on constructing automata:  0.002946429000076023
time spend while search on  2 parts: 22.02881398200043
860
time spending on constructing automata:  0.002751582998826052
time spending on constructing automata:  0.0028735890009556897
time spending on constructing automata:  0.0027805579993582796
time spending on constructing automata:  0.002844829999958165
time spend while search on  4 parts: 27.72060175400111
860
time spending on constructing automata:  0.002750373001617845
time spending on constructing automata:  0.0032692259992472827
time spending on constructing automata:  0.0034328100009588525
time spending on constructing automata:  0.003421127999899909
time spending on constructing automata:  0.003366351998920436
time spending on constructing automata:  0.003436451999732526
time spending on constructing automata:  0.0037879419996897923
time spending on constructing automata:  0.0038604719993600156
tim